# Notebook ICD - 14
## Importamos librarias

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter

### Naive Bayes desde cero
Esta sección implementa la clase NaiveBayesClassifier que incluye dos métodos principales: fit y predict.

El método **init** inicializa las estructuras de datos que almacenarán las probabilidades a priori de cada clase (self.class_priors), así como las probabilidades condicionales de cada atributo dada una clase (self.likelihoods).

El método **fit** se encarga de calcular las probabilidades a priori de las clases a partir de las frecuencias observadas en los datos de entrenamiento y, a continuación, calcula las probabilidades (probabilidades condicionales) aplicando el suavizado de Laplace para evitar valores de probabilidad cero cuando no se ha observado un valor de atributo.

Por último, el método de **predict** toma instancias de prueba, calcula las probabilidades a posteriori de cada clase y asigna la clase con la probabilidad más alta a cada instancia.

In [6]:
class NaiveBayesClassifier:
    def __init__(self):
        self.class_priors = {}  # Prior probabilities of the classes
        self.likelihoods = {}   # Conditional probabilities (likelihoods)
        self.classes = None     # Unique classes in the dataset
        self.features = None    # Features (attributes)
    
    def fit(self, X, y):

        # Get the unique classes and features (attributes)
        self.classes = np.unique(y)
        self.features = X.columns
        total_samples = len(y)  # Total number of training instances
        
        # Estimate prior probabilities (relative frequency of each class)
        class_counts = y.value_counts().to_dict()
        self.class_priors = {cls: (class_counts[cls] / total_samples) for cls in self.classes}
        
        # Initialize conditional probabilities (likelihoods)
        self.likelihoods = {cls: {} for cls in self.classes}
        
        # Calculate the likelihoods (conditional probabilities) for each feature
        for cls in self.classes:
            X_cls = X[y == cls]  # Filter instances where the class is 'cls'
            total_cls_samples = len(X_cls)  # Number of instances per class
            
            # Calculate the likelihoods for each attribute and attribute value
            for feature in self.features:
                feature_counts = X_cls[feature].value_counts().to_dict()  # Frequency of each attribute value
                total_feature_values = len(X[feature].unique())  # Total number of possible attribute values
                
                # Apply Laplace smoothing and calculate the likelihoods
                self.likelihoods[cls][feature] = {
                    value: (feature_counts.get(value, 0) + 1) / (total_cls_samples + total_feature_values)
                    for value in X[feature].unique()
                }
    
    def predict(self, X_test):
        
        results = []
        
        # Iterate over each test instance
        for _, x in X_test.iterrows():
            class_probabilities = {}  # Store the posterior probabilities for each class
            
            # Calculate the posterior probability for each class
            for cls in self.classes:
                # Initialize with the prior probability of the class
                prob = self.class_priors[cls]
                
                # Multiply by the likelihoods (conditional probabilities) of each feature
                for feature in self.features:
                    value = x[feature]
                    prob *= self.likelihoods[cls][feature].get(value, 1 / (len(self.likelihoods[cls][feature]) + len(self.features)))
                
                # Store the calculated probability for the class
                class_probabilities[cls] = prob
            
            # Select the class with the highest posterior probability
            predicted_class = max(class_probabilities, key=class_probabilities.get)
            results.append(predicted_class)
        
        return results

### Ejemplo de aplicación

Se importará el conjunto de datos «Jugar al tenis» para construir un clasificador Naive Bayes que prediga si se va a jugar o no al tenis en función de condiciones meteorológicas como la temperatura, la humedad y el viento. Las 14 instancias disponibles servirán de base de entrenamiento para el modelo, mientras que una nueva instancia, no incluida en el entrenamiento, se utilizará para evaluar su rendimiento y generalización.

In [9]:
data = pd.read_csv('weather.nominal - weather.nominal.csv')

# Define X (features) and y (label)
X = data.iloc[:, :-1]  # All columns except the last one
y = data.iloc[:, -1]  # Last column (label)

# Train the classifier using the Naive Bayes algorithm with the original column names
nb_classifier = NaiveBayesClassifier()
nb_classifier.fit(X, y)

# Create the instance to test: sunny, hot, normal, TRUE
test_instance = pd.DataFrame([{
    'outlook': 'sunny',
    'temperature': 'cool',
    'humidity': 'high',
    'windy': True
}])

# Make the prediction
prediction = nb_classifier.predict(test_instance)
print(f"Prediction for the instance {test_instance.iloc[0].to_dict()}: {prediction[0]}")

Prediction for the instance {'outlook': 'sunny', 'temperature': 'cool', 'humidity': 'high', 'windy': True}: no


### Implementación de Scikit-learn

El algoritmo Naive Bayes es un clasificador probabilístico sencillo y eficiente que asume la independencia condicional entre las características. Aunque esta suposición no siempre se cumple en los datos del mundo real, Naive Bayes suele funcionar extraordinariamente bien en muchas aplicaciones.

#### Clasificador Naive Bayes

El algoritmo Naive Bayes se basa en el Teorema de Bayes, que se expresa como:

[ P(C|X) = \frac{P(X|C)P(C)}{P(X)} ]

donde

(P(C|X)) representa la probabilidad posterior de la clase (C) dados los datos (X),
(P(X|C)) es la probabilidad de los datos dada la clase (C),
(P(C)) es la probabilidad a priori de la clase (C),
(P(X)) es la probabilidad de los datos (que es constante para todas las clases y puede ignorarse a efectos de clasificación).
Bayes ingenuo gaussiano

En el caso del Naive Bayes gaussiano (GaussianNB), el algoritmo asume que las características siguen una distribución gaussiana (normal). La probabilidad de una característica (x_i) dada una clase (C_k) se calcula utilizando la función de densidad de probabilidad de la distribución gaussiana:

[ P(x_i | C_k) = \frac{1}{\sqrt{2\pi \sigma_k^2}} \exp\left(-\frac{(x_i - \mu_k)^2}{2\sigma_k^2}\right) ]

donde:

( \mu_k ) denota la media de la característica (x_i) para la clase (C_k),
( \sigma_k^2 ) es la varianza de la característica (x_i) para la clase (C_k),
( x_i ) representa el valor de la característica para la instancia dada.

### Biblioteca

In [12]:
from sklearn.naive_bayes import GaussianNB

### Datos

In [16]:
df = pd.read_csv(r'weather.numeric.csv')

In [19]:
print (df) 

    Day   Outlook  Temperature  Humidity    Wind   Play
0     1     sunny           85        85    weak  False
1     2     sunny           80        90  strong  False
2     3  overcast           83        86    weak   True
3     4      rain           70        96    weak   True
4     5      rain           68        80    weak   True
5     6      rain           65        70  strong  False
6     7  overcast           64        65  strong   True
7     8     sunny           72        95    weak  False
8     9     sunny           69        70    weak   True
9    10      rain           75        80    weak   True
10   11     sunny           75        70  strong   True
11   12  overcast           72        90  strong   True
12   13  overcast           81        75    weak   True
13   14      rain           71        91  strong  False


In [21]:
# defining the dependent and independent variables
X_train = df[['Outlook', 'Temperature', 'Humidity', 'Wind']]
y_train = df[['Play']]

print(X_train.head())
print(y_train.head())

    Outlook  Temperature  Humidity    Wind
0     sunny           85        85    weak
1     sunny           80        90  strong
2  overcast           83        86    weak
3      rain           70        96    weak
4      rain           68        80    weak
    Play
0  False
1  False
2   True
3   True
4   True


### From categorical to numeric

In [24]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

outlook = X_train.iloc[:,0]
outlook_enc = encoder.fit_transform(outlook)
print(outlook.tolist())
print(outlook_enc)

wind = X_train.iloc[:,3]
wind_enc = encoder.fit_transform(wind)
print(wind.tolist())
print(wind_enc)

['sunny', 'sunny', 'overcast', 'rain', 'rain', 'rain', 'overcast', 'sunny', 'sunny', 'rain', 'sunny', 'overcast', 'overcast', 'rain']
[2 2 0 1 1 1 0 2 2 1 2 0 0 1]
['weak', 'strong', 'weak', 'weak', 'weak', 'strong', 'strong', 'weak', 'weak', 'weak', 'strong', 'strong', 'weak', 'strong']
[1 0 1 1 1 0 0 1 1 1 0 0 1 0]


In [26]:
df_outlook = pd.DataFrame(outlook_enc, columns = ['Outlook'])
df_wind = pd.DataFrame(outlook_enc, columns = ['Wind'])
X_train_num = pd.concat([df_outlook, X_train.iloc[:,1], X_train.iloc[:,2], df_wind], axis=1)
print(X_train_num)

    Outlook  Temperature  Humidity  Wind
0         2           85        85     2
1         2           80        90     2
2         0           83        86     0
3         1           70        96     1
4         1           68        80     1
5         1           65        70     1
6         0           64        65     0
7         2           72        95     2
8         2           69        70     2
9         1           75        80     1
10        2           75        70     2
11        0           72        90     0
12        0           81        75     0
13        1           71        91     1


### Generación del modelo
Gaussian Naive Bayes. GaussianNB implements the Gaussian Naive Bayes algorithm for classification. The likelihood of the features is assumed to be Gaussian.

In [29]:
clf = GaussianNB().fit(X_train_num, y_train)

C:\Users\ingeo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Evaluación

In [32]:
# sunny:2, hot:85, normal:65, strong:0 
new_example = [[2, 60, 65, 1]]
X_test = pd.DataFrame(new_example, columns = ['Outlook', 'Temperature', 'Humidity', 'Wind'])
print(X_test)
clf.predict(X_test)

   Outlook  Temperature  Humidity  Wind
0        2           60        65     1


array([ True])